# Code-to-Code Comparison: Dinwoodie

### National Renewable Energy Laboratory

#### Rob Hammond

##### 31 October 2022

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import DINWOODIE, load_yaml

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base_100pct_reduction",
    "more_ctvs_100pct_reduction",
    "fewer_ctvs_100pct_reduction",
    "more_techs_100pct_reduction",
    "fewer_techs_100pct_reduction",
    "failure_50_100pct_reduction",
    "failure_200_100pct_reduction",
    "no_hlvs_100pct_reduction",
    "no_weather_100pct_reduction",
    "historic_weather_100pct_reduction",
    "manual_resets_only_100pct_reduction",
    "minor_repairs_only_100pct_reduction",
    "medium_repairs_only_100pct_reduction",
    "major_repairs_only_100pct_reduction",
    "major_replacements_only_100pct_reduction",
    "annual_service_only_100pct_reduction",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
    
}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(50)} | {(end - start) / 60:.2f} m")
    
    # Gather the results of interest
    years = sim.metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability_time = sim.metrics.time_based_availability(frequency="project", by="windfarm")
    availability_production = sim.metrics.production_based_availability(frequency="project", by="windfarm")
    cf_net = sim.metrics.capacity_factor(which="net", frequency="project", by="windfarm")
    cf_gross = sim.metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
    power_production = sim.metrics.power_production(frequency="project", by_turbine=False).values[0][0]
    completion_rate = sim.metrics.task_completion_rate(which="both", frequency="project")
    parts = sim.metrics.events[["materials_cost"]].sum().sum()
    techs = sim.metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    total = sim.metrics.events[["total_cost"]].sum().sum()
    
    equipment = sim.metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]].sum().sum()
    fsv = equipment[[el for el in equipment.columns if "Field Support Vessel" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    
    utilization = sim.metrics.service_equipment_utilization(frequency="project")
    hlv_ur = utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]].mean().mean()
    fsv_ur = utilization[[el for el in utilization.columns if "Field Support Vessel" in el]].mean().mean()
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)
    sim.env.cleanup_log_files(log_only=True)

                   dinwoodie_base_100pct_reduction | 0.61 m
              dinwoodie_more_ctvs_100pct_reduction | 0.61 m
             dinwoodie_fewer_ctvs_100pct_reduction | 0.59 m
             dinwoodie_more_techs_100pct_reduction | 0.57 m
            dinwoodie_fewer_techs_100pct_reduction | 0.58 m
             dinwoodie_failure_50_100pct_reduction | 0.64 m
            dinwoodie_failure_200_100pct_reduction | 0.66 m
                dinwoodie_no_hlvs_100pct_reduction | 0.63 m
             dinwoodie_no_weather_100pct_reduction | 0.68 m
       dinwoodie_historic_weather_100pct_reduction | 0.56 m
     dinwoodie_manual_resets_only_100pct_reduction | 0.63 m
     dinwoodie_minor_repairs_only_100pct_reduction | 0.66 m
    dinwoodie_medium_repairs_only_100pct_reduction | 0.64 m
     dinwoodie_major_repairs_only_100pct_reduction | 0.67 m
dinwoodie_major_replacements_only_100pct_reduction | 0.67 m
    dinwoodie_annual_service_only_100pct_reduction | 0.87 m


In [5]:
# Save the results
# pickled dictionary format
with open(DINWOODIE / "outputs" / "results_dict_100pct_reduction_v0.5.2.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(DINWOODIE / "outputs" / "results_data_100pct_reduction_v0.5.2.csv", index_label="result")

In [6]:
results_df

,base_100pct_reduction,more_ctvs_100pct_reduction,fewer_ctvs_100pct_reduction,more_techs_100pct_reduction,fewer_techs_100pct_reduction,failure_50_100pct_reduction,failure_200_100pct_reduction,no_hlvs_100pct_reduction,no_weather_100pct_reduction,historic_weather_100pct_reduction,manual_resets_only_100pct_reduction,minor_repairs_only_100pct_reduction,medium_repairs_only_100pct_reduction,major_repairs_only_100pct_reduction,major_replacements_only_100pct_reduction,annual_service_only_100pct_reduction
availability - time based,windfarm 0 0.96,windfarm 0 0.98,windfarm 0 1.00,windfarm 0 0.97,windfarm 0 1.00,windfarm 0 0.98,windfarm 0 0.97,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 0.98,windfarm 0 0.98
availability - production based,windfarm 0 0.96,windfarm 0 0.98,windfarm 0 1.00,windfarm 0 0.97,windfarm 0 1.00,windfarm 0 0.98,windfarm 0 0.97,windfarm 0 1.00,windfarm 0 0.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 0.98,windfarm 0 0.99
capacity factor - net,windfarm 0 0.46,windfarm 0 0.46,windfarm 0 0.47,windfarm 0 0.46,windfarm 0 0.47,windfarm 0 0.46,windfarm 0 0.46,windfarm 0 0.47,windfarm 0 0.00,windfarm 0 0.48,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.46,windfarm 0 0.47
capacity factor - gross,windfarm 0 0.48,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.00,windfarm 0 0.48,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47,windfarm 0 0.47
power production,"9,657,337,634.50","9,335,609,771.50","9,571,233,544.00","9,332,757,608.50","9,569,943,637.50","9,335,641,105.50","9,332,940,988.00","9,570,851,949.50",0.00,"8,086,663,481.50","9,572,237,537.00","9,571,753,632.50","9,568,496,126.00","9,569,019,848.00","9,356,767,763.50","9,433,592,991.00"
task completion rate,windfarm 0 0.96,windfarm 0 0.97,windfarm 0 1.00,windfarm 0 0.97,windfarm 0 1.00,windfarm 0 0.97,windfarm 0 0.97,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 1.00,windfarm 0 0.97,windfarm 0 0.96,windfarm 0 0.90
annual direct O&M cost,3.53,7.64,4.31,5.58,3.01,4.31,4.32,4.29,4.32,3.89,4.31,4.29,5.22,4.88,12.01,4.50
annual vessel cost,1.92,2.84,1.74,1.73,1.72,1.74,1.76,1.73,1.75,1.74,1.75,1.72,1.91,1.98,8.01,1.76
ctv cost,1.92,2.84,1.74,1.73,1.72,1.74,1.76,1.73,1.75,1.74,1.75,1.72,1.91,1.63,1.69,1.76
fsv cost,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.35,0.00,0.00
